In [ ]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp
import numpy as np
from sklearn.linear_model import LinearRegression
from datetime import datetime as dt
from google.cloud import bigquery, secretmanager

In [ ]:
def set_secrets_as_env(split_on=':'):
    secrets = secretmanager.SecretManagerServiceClient()
    resource_name = f"projects/knada-gcp/secrets/vebjorn-rekkebo-bac1/versions/latest"
    secret = secrets.access_secret_version(name=resource_name)
    secrets = secret.payload.data.decode('UTF-8')
    for secret in secrets.splitlines():
        key, value = secret.split(split_on)
        os.environ[key] = value

In [ ]:
set_secrets_as_env()

In [ ]:
Client = bigquery.Client('nais-analyse-prod-2dcc')

In [ ]:
df_bq = Client.query("""
select EXTRACT(year from dato) as year, EXTRACT(month from dato) as month,  EXTRACT(ISOWEEK from dato) as week,
case when EXTRACT(month from dato) in (11, 12) then EXTRACT(year from dato) + 1
else EXTRACT(year from dato) end as fin_year,
env, service_description, sum(calculated_cost) as calculated_cost
from `nais-analyse-prod-2dcc.navbilling.cost_breakdown_total`
where dato >= '2021-11-01'
and tenant in ('nav', 'dev-nais', 'example')
group by year, month, week, fin_year, env, service_description
order by year asc, month asc, week asc;
""").result().to_dataframe()

In [ ]:
FIRST_YEAR = 2021
FIRST_WEEK = 44 # 1. november 2021


def agg_week(row):
    if row.week >= 52 and row.month == 1:
        return row.week + 52 * (row.year - FIRST_YEAR - 1) - (FIRST_WEEK - 1)
    else:
        return row.week + 52 * (row.year - FIRST_YEAR) - (FIRST_WEEK - 1)
    
    
def unagg_week(week):
    week_num = (week + (FIRST_WEEK - 1)) % 52
    year = (week + (FIRST_WEEK - 1)) // 52 + FIRST_YEAR
    if week_num == 0:
        return f"{year-1} - {str(52).zfill(2)}"
    else:
        return f"{year} - {str(week_num).zfill(2)}"

In [ ]:
df = df_bq.copy()

df["week_padded"] = df["week"].apply(str).apply(lambda x: x.zfill(2))
df["year_week"] = df[["year", "week_padded"]].apply(lambda row: "-".join(row.values.astype(str)), axis=1)
df["n_week"] = df.apply(lambda row: agg_week(row), axis=1)
df["month_padded"] = df["month"].apply(str).apply(lambda x: x.zfill(2))
df["year_month"] = df[["year", "month_padded"]].apply(lambda row: "-".join(row.values.astype(str)), axis=1)

df_2022 = df[df.fin_year == 2022]
df_2023 = df[df.fin_year == 2023]

In [ ]:
cols = ["week", "n_week", "service_description"]

current_n_week = df.n_week.max()
df_service = df[df.n_week < current_n_week].groupby(cols, as_index=False).calculated_cost.sum().sort_values("n_week").reset_index(drop=True)
df_service["year_week"] = df_service.n_week.apply(unagg_week)

In [ ]:
n_top = 2
top_services = set(df_service
                   .groupby("service_description", as_index=False)
                   .calculated_cost
                   .sum()
                   .sort_values("calculated_cost", ascending=False)
                   .iloc[:n_top]
                   .service_description
                  )
top_services

In [ ]:
def group_services(service):
    if service in top_services:
        return service
    else:
        return "Other services"

In [ ]:
# 'Cloud Dialogflow API' ødelegger rekkefølgen på stolpene så vi tar den bort. Det er snakk om mindre enn en euro.
df_plot_services = df_service[df_service.service_description != 'Cloud Dialogflow API']
fig_tot = px.bar(df_plot_services, "year_week", "calculated_cost", color="service_description", barmode="stack", labels={"year_week":"År - uke", "calculated_cost":"Kostnad (€)"})
#fig_tot.update_xaxes(dtick=5, tick0='2021 - 44')
fig_tot.update_xaxes(categoryorder='array', categoryarray=df_service.year_week.unique() )

fig_tot.show()

In [ ]:
df_tot = df_service.groupby(["n_week"], as_index=False).calculated_cost.sum()
px.line(df_tot, "n_week", "calculated_cost")

# Regresjon

In [ ]:
weeks_to_look_at = 15 # ENDRE HER

def make_training_data(df_service, current_n_week=current_n_week, weeks_to_look_at=weeks_to_look_at):

    df_services_grouped = df_service.copy()
    df_services_grouped["service_description"] = df_services_grouped.service_description.apply(lambda x: group_services(x))
    df_services_grouped["year_week"] = df_services_grouped.n_week.apply(unagg_week)
    df_s = df_services_grouped.groupby(["service_description", "n_week"], as_index=False).calculated_cost.sum()

    start_n_week = current_n_week - weeks_to_look_at 
    df_s = df_s[(df_s.n_week >= start_n_week) & (df_s.n_week < current_n_week)]

    x_s = df_s[["service_description", "n_week"]]
    x_s = pd.get_dummies(data=x_s)
    y_s = df_s[["calculated_cost"]]
    
    return x_s, y_s, df_services_grouped

In [ ]:
x_s, y_s, df_services_grouped = make_training_data(df_service, current_n_week, 15)

In [ ]:
#x_s

In [ ]:
mod = LinearRegression().fit(x_s.values, y_s.values)

In [ ]:
mod.intercept_

In [ ]:
def make_week_vars(current_n_week=current_n_week):
    previous_n_week = current_n_week - 1
    n_weeks = 52*2 - previous_n_week
    n_services = x_s.shape[1] - 1
    return previous_n_week, n_weeks, n_services

In [ ]:
previous_n_week, n_weeks, n_services = make_week_vars(current_n_week)
print(previous_n_week, n_weeks, n_services)

In [ ]:
def predict_cost(mod, current_n_week, n_weeks, n_services):
    cost = np.zeros([n_weeks + 1, n_services])
    for week in range(current_n_week, 2*52+1):
        for service in range(n_services):
            z = np.zeros([n_services])
            z[service] = 1
            x1 = np.append([week], z).reshape(1,n_services+1)
            cost[week-current_n_week, service] += mod.predict(x1)[0]

    # Last week of October 2023 has only two days.
    last_week = 2*52+1
    z = np.zeros([n_services])
    for service in range(n_services):
        z[service] = 1
        x1 = np.append([last_week], z).reshape(1,n_services+1)
        cost[last_week-current_n_week, service] += mod.predict(x1)[0] * 2/7
    return cost

In [ ]:
cost = predict_cost(mod, current_n_week, n_weeks, n_services)

In [ ]:
def cost_to_df(cost, x_s, current_n_week):
    df_cost = pd.DataFrame(cost)
    df_cost.columns = pd.Series(x_s.columns[1:]).apply(lambda x: x[20:])

    df_cost["n_week"] = df_cost.index + current_n_week
    return df_cost

In [ ]:
df_cost = cost_to_df(cost, x_s, current_n_week)

In [ ]:
df_cost.head()

In [ ]:
def make_df_plot(df_cost, current_n_week):
    df_plot = df_cost.sum(axis=1).reset_index(name="cost")
    df_plot["year_week"] = df_plot["index"] + current_n_week
    df_plot["year_week"] = df_plot["year_week"].apply(unagg_week)
    df_plot["index"] = df_plot["index"] + current_n_week - 52
    df_plot = df_plot.rename(columns={"index":"week"})
    return df_plot

In [ ]:
df_plot = make_df_plot(df_cost, current_n_week)
df_plot.head()

In [ ]:
df_2023.tail()

In [ ]:
def compute_total_costs(df_plot, current_n_week):
    kostnad_i_fjor = df_2022.calculated_cost.sum()
    kostnad_hittil = df_2023[(df_2023.n_week < current_n_week)].calculated_cost.sum()

    forventet_resten = df_plot.cost.sum()
    forventet_totalt = kostnad_hittil + forventet_resten
    return kostnad_i_fjor, kostnad_hittil, forventet_resten, forventet_totalt

In [ ]:
kostnad_i_fjor, kostnad_hittil, forventet_resten, forventet_totalt = compute_total_costs(df_plot, current_n_week)
print(f"Kostnad i fjor: {int(kostnad_i_fjor)} € \nKostnad hittil i år: {int(kostnad_hittil)} € \nForventet gjenstående kostnad i år: {int(forventet_resten)} € \nForventet totalkostnad i år: {int(forventet_totalt)} €")

In [ ]:
fig_numbers = sp.make_subplots(rows=2, cols=2,
                                 specs=[[{"type": "indicator"} ,{"type": "indicator"}],
                                       [{"type": "indicator"},{"type": "indicator"}]])

fig_numbers.add_trace(go.Indicator(
        mode = 'number', 
        value = int(kostnad_hittil),
        number = {'valueformat': ',', 'suffix' :' €'},
        title = {'text': '{0}<br><span style=\'font-size:0.7em;color:gray\'>{1}</span>'.format("","Kostnad hittil i år (eksludert inneværende uke)")}
    ), row = 1, col = 1)
fig_numbers.add_trace(go.Indicator(
        mode = 'number', 
        value = int(forventet_resten),
        number = {'valueformat': ',', 'suffix' :' €'},
        title = {'text': '{0}<br><span style=\'font-size:0.7em;color:gray\'>{1}</span>'.format("","Forventet kostnad resten av året")}
    ), row = 2, col = 1)
fig_numbers.add_trace(go.Indicator(
        mode = 'number', 
        value = int(forventet_totalt),
        number = {'valueformat': ',', 'suffix' :' €'},
        title = {'text': '{0}<br><span style=\'font-size:0.7em;color:gray\'>{1}</span>'.format("","Forventet totalkostnad i år")}
    ), row = 1, col = 2)
fig_numbers.add_trace(go.Indicator(
        mode = 'number', 
        value = int(kostnad_i_fjor),
        number = {'valueformat': ',', 'suffix' :' €'},
        title = {'text': '{0}<br><span style=\'font-size:0.7em;color:gray\'>{1}</span>'.format("","Totalkostnad i fjor")}
    ), row = 2, col = 2)

In [ ]:
df_tot["year_week"] = df_tot.n_week.apply(unagg_week)
df_tot.head()

In [ ]:
fig_forecast = px.bar(df_tot, "year_week", "calculated_cost", title="Kostnad per uke november 2021 - oktober 2023", color_discrete_sequence=["rgba(0, 86, 180, 1)"], labels={"year_week":"Uke", "calculated_cost":"Kostnad (€)"})
fig_forecast.update_traces(name='Faktisk kostnad', showlegend = True)
fig_forecast.add_vline(x='23 - 01', line_dash="dash", line_color="gray")
fig_forecast.add_traces(go.Bar(x=df_plot.year_week, y=df_plot.cost, name="Forventet kostnad", marker_color="rgba(153, 195, 255, 1)"))


In [ ]:
df_cost.head()

In [ ]:
df_pred = pd.melt(df_cost, id_vars=['n_week'])
df_pred.head()

df_pred["year_week"] = df_pred.n_week.apply(lambda x: unagg_week(x))

In [ ]:
services = list(df_services_grouped.service_description.unique())
services.sort()
service_to_color = dict(zip(services, px.colors.qualitative.Alphabet*2))
px.colors.qualitative.Alphabet

plot_data=[go.Bar(x=df_services_grouped[df_services_grouped.service_description == services[i]].n_week,
                  y=df_services_grouped[df_services_grouped.service_description == services[i]].calculated_cost,
                  marker_color=service_to_color[services[i]],
                  name=services[i])
                  for i in range(len(services))]


In [ ]:
print(services)

In [ ]:
fig_2 = px.bar(df_pred, "n_week", "value", color="variable", barmode="stack", color_discrete_map=service_to_color)
fig_2.add_traces(data=plot_data)
#fig_2.update_layout({"height": 500})
fig_2.show()

# Historikk på forventede utgifter

In [ ]:
first_n_week = 53
predictions = np.zeros([current_n_week - first_n_week + 1])

for n_week in range(first_n_week, current_n_week + 1):
    #n_week = 55
    x_s, y_s, df_services_grouped = make_training_data(df_service, n_week, 15)
    #print(x_s.shape, y_s.shape)
    mod = LinearRegression().fit(x_s.values, y_s.values)
    previous_n_week, n_weeks, n_services = make_week_vars(n_week)
    #print(previous_n_week, n_weeks, n_services)
    cost = predict_cost(mod, n_week, n_weeks, n_services)
    df_cost = cost_to_df(cost, x_s, n_week)
    df_plot = make_df_plot(df_cost, n_week)
    kostnad_i_fjor, kostnad_hittil, forventet_resten, forventet_totalt = compute_total_costs(df_plot, n_week)
    #print(f"Kostnad i fjor: {int(kostnad_i_fjor)} € \nKostnad hittil i år: {int(kostnad_hittil)} € \nForventet gjenstående kostnad i år: {int(forventet_resten)} € \nForventet totalkostnad i år: {int(forventet_totalt)} €")
    predictions[n_week - first_n_week] = forventet_totalt

In [ ]:
np.linspace(predictions.shape[0]-1, 0, predictions.shape[0])

In [ ]:
predictions

In [ ]:
fig_historic_predictions = px.line(x=np.linspace(predictions.shape[0]-1, 0, predictions.shape[0]), y=predictions,
              title="Totalkostnad i 2023 predikert på ulike tidspunkt",
              labels={"x":"Uker siden", "y":"Forventet kostnad 2023 (€)"})
fig_historic_predictions.update_xaxes(autorange="reversed")

# Raskest voksende utgifter

In [ ]:
df_days = Client.query("""
select dato, env, service_description, sum(calculated_cost) calculated_cost from `nais-analyse-prod-2dcc.navbilling.cost_breakdown_total`
where current_date() - 22 <= dato and dato < current_date() - 1
and tenant in ('nav', 'dev-nais', 'example')
group by dato, env, service_description;
""").result().to_dataframe()

In [ ]:
df_days[(df_days["service_description"] == 'Cloud SQL') & (df_days["env"] == 'prod')].sort_values("dato").reset_index(drop=True)

In [ ]:
from datetime import datetime, timedelta
# Ser ikke på gårsdagen fordi den ikke er ferdig oppdatert på kjøretidspunktet
df_previous_7_days = df_days[df_days.dato >= (datetime.today() - timedelta(8)).date()]
df_earlier = df_days[df_days.dato < (datetime.today() - timedelta(8)).date()]

In [ ]:
df_earlier.dato.unique()

In [ ]:
df_previous_grouped = df_previous_7_days.groupby(["service_description", "env"], as_index=False).calculated_cost.mean()
df_earlier_grouped = df_earlier.groupby(["service_description", "env"], as_index=False).calculated_cost.mean()

In [ ]:
df_change = df_previous_grouped.merge(df_earlier_grouped,
                              on=["service_description", "env"],
                              suffixes=["", "_mean"])

In [ ]:
df_change.head()

In [ ]:
df_change["growth_euro"] = df_change.calculated_cost - df_change.calculated_cost_mean
df_change["growth_percent"] = df_change.growth_euro / df_change.calculated_cost_mean


In [ ]:
df_highest_percent = df_change.sort_values("growth_percent", ascending=False).reset_index(drop=True)
df_highest_percent = df_highest_percent[df_highest_percent.calculated_cost > 10]
df_highest_euro = df_change.sort_values("growth_euro", ascending=False).reset_index(drop=True)

In [ ]:
df_highest_percent.head()

In [ ]:
df_highest_euro.head()

In [ ]:
labels = {"service_description":"Produkt", "growth_percent":"Endring (%)", "growth_euro":"Endring (€)"}

fig_percent = px.bar(df_highest_percent[df_highest_percent.env == "prod"], "service_description", "growth_percent", height=600, hover_data=["service_description", "growth_euro", "growth_percent"], labels=labels, title="Kostnadsendring i prod (%)")
fig_percent.update_layout(yaxis={'tickformat':".0%"})
fig_euro = px.bar(df_highest_euro[df_highest_euro.env == "prod"], "service_description", "growth_euro", height=600, custom_data=["growth_percent"], labels=labels, title="Kostnadsendring i prod (€)")
fig_euro.update_traces(hovertemplate="<br>".join([
        "%{x}",
        "Endring (€): %{y}",
        "Endring (%): %{customdata[0]: .0%}"
    ]))

fig_percent_dev = px.bar(df_highest_percent[df_highest_percent.env == "dev"], "service_description", "growth_percent", height=600, hover_data=["service_description", "growth_euro", "growth_percent"], labels=labels, title="Kostnadsendring i dev (%)")
fig_percent_dev.update_layout(yaxis={'tickformat':".0%"})
fig_euro_dev = px.bar(df_highest_euro[df_highest_euro.env == "dev"], "service_description", "growth_euro", height=600, custom_data=["growth_percent"], labels=labels, title="Kostnadsendring i dev (€)")
fig_euro_dev.update_traces(hovertemplate="<br>".join([
        "%{x}",
        "Endring (€): %{y}",
        "Endring (%): %{customdata[0]: .0%}"
    ]))

figs = [fig_percent, fig_euro, fig_percent_dev, fig_euro_dev]
for fig in figs:
    fig.show()

# Datastory

In [ ]:
import datastory
ds = datastory.DataStory("Kostnader i GCP og Aiven")

ds.header("Kostnader i GCP og Aiven")
ds.markdown("Det er strammere tider og derfor større behov for å ha kontroll på store utgiftsposter. Her har vi prøvd å predikere hvor mye penger Nav kommer til å bruke på GCP og Aiven i 2023.")
ds.header("Nøkkeltall")
ds.markdown(f"""Forventninger er basert på utvikling siste {weeks_to_look_at} uker. 
            Merk at det er vanskelig å spå framtiden, og forventningene er basert på en lite avansert modell. 
            Derfor er det viktig å være klar over at forventede kostnader sannsynligvis ikke treffer presist på faktisk kostnad. 
            Modellen kan potensielt bomme med flere hundre tusen euro over et år.
            """)
ds.markdown("""Midlertidige kilder til unøyaktighet i disse tallene: 
            - Foreløpig er nais-kostnadene til SSB inkludert i datagrunnlaget. Dette utgjør en svært liten andel av den totale regninga.
            - Aivenkostnader er basert på en fast dollarkurs for omregning til euro. Denne avviker tidvis betydelig fra reell kurs. Det er heller ikke her snakk om ekstreme summer.
            """)
ds.markdown("""NB! Et år er her definert fra 1. november til 31. oktober for å enklere kunne sammenligne med regnskap.""")
ds.plotly(fig_numbers.to_json())

ds.header("Kostnader per service siden 1.november 2021.")
ds.markdown("Merk at det mangler noe data fra uke 2 i 2022, slik at totalen denne uka ser lavere ut enn den var.")
ds.plotly(fig_tot.to_json())

ds.header(f"Forventet utvikling i kostnader ut oktober 2023 basert på utvikling siste {weeks_to_look_at} uker")
ds.markdown("1.november 2021 var mandag uke 44. Fem av dagene i uke 44 2023 er i november, og regnes derfor ikke med.")
ds.plotly(fig_forecast.to_json())

ds.header(f"Historisk prediksjon")
ds.markdown(f"""Figuren viser hva den forventede totalkostnaden for budsjettåret 2023 var på ulike tidspunkt.
            Formålet er å få et bilde på hvor stor usikkerhet det var i prediksjonen på starten av året og hvordan den blir mindre etter som vi nærmer oss slutten av året.  
            X-aksen viser hvor mange uker det er siden hver prediksjon ble gjort.
            Det vil si at 0 angir siste prediksjon av totalkostnad for 2023 og første prediksjon ble gjort i starten av oktober 2022.""")
ds.plotly(fig_historic_predictions.to_json())

ds.header("Største endringer i kostnad i prod.", level=2)
ds.markdown("For å ikke få store overraskelser må vi følge med på store endringer i kostnad for de ulike tjenestene.")
ds.markdown("Vi sammenligner siste 7 dager (til og med to dager siden) med to foregående uker. Eksempel: Hvis datoen i dag var 09.02. ville vi sett på perioden 01.02.-07.02. (7 dager) sammenlignet med perioden 18.01.-31.01. (14 dager).")
ds.plotly(fig_euro.to_json())
ds.markdown("For prosentvis endring ser vi kun på produkter som i snitt koster minst 5€ per dag.")
ds.plotly(fig_percent.to_json())
ds.header("Største endringer i kostnad i dev. Siste 7 dager sammenlignet med to foregående uker.", level=2)
ds.plotly(fig_euro_dev.to_json())
ds.markdown("For prosentvis endring ser vi kun på produkter som i snitt koster minst 5€ per dag.")
ds.plotly(fig_percent_dev.to_json())

In [ ]:
ds.update(url="https://nada.intern.nav.no/api", token=os.environ["KOSTNAD_STORY_TOKEN"])